In [1]:
import uproot
import numpy as np
import awkward
import concurrent.futures
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
import plotly.graph_objects as go
import plotly.express as px
import copy
import pandas as pd

In [2]:
# Example Code for Creating Lines with Multiple Colors
df = pd.DataFrame(dict(
    X=[0,1,2,3,1,4], 
    Y=[0,2,3,1,1,2], 
    Z=[0,3,1,2,1,3],
    color=["a", "a", "a", "a", "b", "b"]
))

fig = px.line_3d(df, x='X', y='Y', z='Z', color="color")

df2 = {}
df2['X'] = [0,2,3]
df2['Y'] = [0,1,3]
df2['Z'] = [0,1,2]

fig.add_trace(go.Scatter3d(x=df2["X"], y=df2["Y"], z=df2['Z'], mode='markers'))
fig.show()

In [7]:
# LOAD THE DATA #
file = 'tough_kaons_skimmed_hough_v2.root'

branches = ['ax','ay','az','bx','by','bz','nlines','hitX','hitY','hitZ','pdgID','pdg_assoc', 'x_assoc','y_assoc','z_assoc',
'electronX', 'electronY', 'electronZ', 'photonX', 'photonY', 'photonZ', 'distFromETraj', 'distFromPTraj']

t = uproot.open(file)['Events']
table = t.arrays(expressions=branches)
tree = {}
for branch in branches:
    tree[branch] = table[branch]

In [10]:
none = 0
for i in tree['nlines']:
    if i == 0:
        none += 1
print(none)

4


In [17]:
event = 1
x = []
y = []
z = []
colors = []

# Add Tracks
for track in range(1):
    track = 0
    for hit in range(len(tree['ax'][event])):
        x.append(tree['ax'][event][track])
        x.append(tree['ax'][event][track] + 200*tree['bx'][event][track])
        x.append(tree['ax'][event][track] - 200*tree['bx'][event][track])
        y.append(tree['ay'][event][track])
        y.append(tree['ay'][event][track] + 200*tree['by'][event][track])
        y.append(tree['ay'][event][track] - 200*tree['by'][event][track])
        z.append(tree['az'][event][track])
        z.append(tree['az'][event][track] + 200*tree['bz'][event][track])
        z.append(tree['az'][event][track] - 200*tree['bz'][event][track])
        colors.append('track')
        colors.append('track')
        colors.append('track')


# Add Electron Trajectory
'''
for hit in range(len(tree['electronZ'][event])):
    x.append(tree['electronX'][event][hit])
    y.append(tree['electronY'][event][hit])
    z.append(tree['electronZ'][event][hit])
    colors.append('electron trajectory')

# Add Photon Trajectory
for hit in range(len(tree['photonX'][event])):
    x.append(tree['photonX'][event][hit])
    y.append(tree['photonY'][event][hit])
    z.append(tree['photonZ'][event][hit])
    colors.append('photon trajectory')
''' 
hits = {}
xHit = []
yHit = []
zHit = []
colorHit = []
# Add Track Hits
for track in range(len(tree['x_assoc'][event])):
    trackNum = track + 1
    for hit in range(len(tree['x_assoc'][event][track])):
        xHit.append(tree['x_assoc'][event][track][hit])
        yHit.append(tree['y_assoc'][event][track][hit])
        zHit.append(tree['z_assoc'][event][track][hit])
        colorHit.append('track')
hits['X'] = xHit
hits['Y'] = yHit
hits['Z'] = zHit
hits['color'] = colorHit

hits = {}
xHit = []
yHit = []
zHit = []
pdgHit = []

# Add Track Hits
for hit in range(len(tree['hitX'][event])):
    

    particle = abs(tree['pdgID'][event][hit])
    if(particle == 321 or particle == 13):
        pdgHit.append('MIP')
        xHit.append(tree['hitX'][event][hit])
        yHit.append(tree['hitY'][event][hit])
        zHit.append(tree['hitZ'][event][hit])

for hit in range(len(tree['hitX'][event])):

    particle = abs(tree['pdgID'][event][hit])
    if not(particle == 321 or particle == 13):
        pdgHit.append('Other')
        xHit.append(tree['hitX'][event][hit])
        yHit.append(tree['hitY'][event][hit])
        zHit.append(tree['hitZ'][event][hit])

hits['X'] = xHit
hits['Y'] = yHit
hits['Z'] = zHit
hits['PDGID'] = pdgHit

lines = pd.DataFrame(dict(X=x,Y=y,Z=z,color=colors))
fig1 = px.line_3d(lines, x='X', y='Y', z='Z', color="color")

fig2 = px.scatter_3d(x=hits['X'], y=hits['Y'], z=hits['Z'],color=hits['PDGID'])

fig3= go.Figure(data=fig1.data + fig2.data)

fig3.update_layout(
    scene = dict(
        xaxis = dict(nticks=6, range=[-200,200],),
                     yaxis = dict(nticks=6, range=[-200,200],),
                     zaxis = dict(nticks=10, range=[240,500],),),
    autosize=False,
    margin=dict(r=20, l=10, b=10, t=10))

fig3.update_traces(marker=dict(size=2),selector=dict(mode='markers'))

fig3.show()

print(tree['pdgID'][event])

[321, 321, 11, 321, 11, 321, 2212, -211, -11, ... 321, 321, 11, 11, 11, 11, -11, 321]
